In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-ec54fbe500fc3b5c.parquet', 'validation': 'data/validation-00000-of-00001-3cf888b12fff1dd6.parquet'}
df_original = pd.read_parquet("hf://datasets/lucadiliello/newsqa/" + splits["validation"])

df = df_original[['context', 'question', 'answers']].copy()
df['answers'] = df['answers'].apply(lambda x: x[0] if x else None)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
df['answers']

,answers
0,three different videos
1,"getting his chest waxed,"
2,environmental
3,his chest
4,Harrison Ford
...,...
4207,give detainees greater latitude in selecting l...
4208,friends
4209,Hilton Head Island
4210,"March 3,"


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [23]:
'''
def answer_question(context, question):
    inputs = tokenizer(context, question, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index])
    )
    return answer

    last_hidden_state = outputs.last_hidden_state
    token_logits = last_hidden_state.mean(dim=2)
    predicted_token_ids = torch.argmax(token_logits, dim=-1)
    generated_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
    return generated_text

In [3]:
def generate_answer(context, question, max_length=50):
    input_text = f"question: {question}  context: {context}"

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=3,
            early_stopping=True
        )


    answer = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return answer

In [4]:
df["predicted_answer"] = df.apply(
    lambda row: generate_answer(row["context"], row["question"]),
    axis=1
)

In [5]:
df

,context,question,answers,predicted_answer
0,(CNN) -- What could be more powerful than the ...,What will be nominated?,three different videos,Iron Eyes Cody
1,(CNN) -- What could be more powerful than the ...,What does the Harrison Ford video feature?,"getting his chest waxed,",his chest waxed
2,(CNN) -- What could be more powerful than the ...,What videos will you send?,environmental,iReport
3,(CNN) -- What could be more powerful than the ...,What is Ford getting waxed?,his chest,his chest
4,(CNN) -- What could be more powerful than the ...,Who got his chest waxed?,Harrison Ford,Harrison Ford
...,...,...,...,...
4207,WASHINGTON (CNN) -- President Obama on Friday ...,What will the new system include?,give detainees greater latitude in selecting l...,greater latitude in selecting legal representa...
4208,NEW YORK (CNN) -- John and Elizabeth Calvert e...,who are growing?,friends,John and Elizabeth Calvert
4209,NEW YORK (CNN) -- John and Elizabeth Calvert e...,Where did the couple live?,Hilton Head Island,Hilton Head Island
4210,NEW YORK (CNN) -- John and Elizabeth Calvert e...,when was the last seen of John and Elizabeth?,"March 3,","March 3, 2008"


In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `baseline-output` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authe

In [6]:
from huggingface_hub import notebook_login

In [45]:
notebook_login()

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import pandas as pd
from datasets import Dataset
hf_dataset = Dataset.from_pandas(df)

In [11]:
hf_dataset.push_to_hub("baseline-dataset", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Shannnh/baseline-dataset/commit/21c9330d8fdc07bbe02545779f3935e4af693776', commit_message='Upload dataset', commit_description='', oid='21c9330d8fdc07bbe02545779f3935e4af693776', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Shannnh/baseline-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Shannnh/baseline-dataset'), pr_revision=None, pr_num=None)